In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape,Flatten
from keras.layers import BatchNormalization, LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

In [2]:
import ssl
import urllib.request

ssl._create_default_https_context = ssl._create_unverified_context

(X_train, _), (_,_) = mnist.load_data()

11490434/11490434 [==============================] - 41s 4us/step


In [10]:
def build_generator():
    model = Sequential()
    model.add(Dense(256,input_dim = 100))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(784,activation='tanh'))
    model.add(Reshape((28,28,1)))
    return model

generator = build_generator()

In [11]:
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha = 0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    return model
discriminator = build_discriminator()
discriminator.compile(optimizer = Adam(0.0002, 0.5), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
discriminator.trainable = False

gan_input = Input(shape=(100, ))
generated_image = generator(gan_input)
gan_output = discriminator(generated_image)

gan = Model(gan_input, gan_output)
gan.compile(optimizer = Adam(0.0002, 0.5),loss = 'binary_crossentropy')

def train_gan(epochs, batch_size = 128):
    X_train, _ = mnist.load_data()
    X_train = (X_train[0].astype(np.float32) - 127.5) /  127.5
    X_train = np.expand_dims(X_train, axis =3)
    
    real = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_images = X_train[idx]
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        generated_images = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(real_images, real)
        d_loss_fake = discriminator.train_on_batch(generated_images, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, real)
        
        if epoch % 100 == 0:
            print(f"{epoch}[D loss: {d_loss[0]}, acc.: {100*d_loss[1]}][G loss:{g_loss}]")
            save_images(epoch)

In [17]:
def save_images(epoch):
    r,c = 5,5
    noise = np.random.normal(0,1,(r*c,100))
    generated_images = generator.predict(noise)
    
    generated_images = 0.5 * generated_images + 0.5
    
    fig,axs = plt.subplots(r,c)
    count = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(generated_images[count, :, :, 0], cmap ='gray')
            axs[i, j].axis('off')
            count += 1
    fig.savefig(f"gan_images_{epoch}.png")
    plt.close()

train_gan(epochs = 10000, batch_size = 64)

2/2 [==============================] - 0s 4ms/step
0[D loss: 0.3698130501434207, acc.: 70.3125][G loss:0.6116334795951843]
2/2 [==============================] - 0s 4ms/step
100[D loss: 0.007253856165334582, acc.: 100.0][G loss:4.898748397827148]
2/2 [==============================] - 0s 4ms/step


2/2 [==============================] - 0s 4ms/step
200[D loss: 0.030777394771575928, acc.: 100.0][G loss:4.302901268005371]
2/2 [==============================] - 0s 4ms/step
300[D loss: 0.3130642995238304, acc.: 84.375][G loss:2.96044921875]
2/2 [==============================] - 0s 4ms/step


2/2 [==============================] - 0s 4ms/step
400[D loss: 0.7039513885974884, acc.: 42.96875][G loss:0.6762176752090454]
2/2 [==============================] - 0s 5ms/step


2/2 [==============================] - 0s 3ms/step
500[D loss: 0.6486066579818726, acc.: 50.0][G loss:0.6912078857421875]
2/2 [==============================] - 0s 4ms/step
600[D loss: 0.647762343287468, acc.: 54.6875][G loss:0.7309393286705017]
2/2 [==============================] - 0s 4ms/step


2/2 [==============================] - 0s 4ms/step
700[D loss: 0.6024916172027588, acc.: 67.96875][G loss:0.8005037307739258]
2/2 [==============================] - 0s 3ms/step


...

2/2 [==============================] - 0s 4ms/step
9300[D loss: 0.6519144177436829, acc.: 61.71875][G loss:0.9281726479530334]
2/2 [==============================] - 0s 5ms/step


2/2 [==============================] - 0s 4ms/step
9400[D loss: 0.7019370198249817, acc.: 58.59375][G loss:0.9015482664108276]
2/2 [==============================] - 0s 4ms/step
9500[D loss: 0.6354238986968994, acc.: 64.84375][G loss:0.8751258850097656]
2/2 [==============================] - 0s 5ms/step


2/2 [==============================] - 0s 4ms/step
9600[D loss: 0.659547746181488, acc.: 59.375][G loss:0.9287635087966919]
2/2 [==============================] - 0s 4ms/step


2/2 [==============================] - 0s 4ms/step
9700[D loss: 0.6419765651226044, acc.: 62.5][G loss:0.9442354440689087]
2/2 [==============================] - 0s 4ms/step
9800[D loss: 0.6232584118843079, acc.: 67.96875][G loss:0.8760768175125122]
2/2 [==============================] - 0s 4ms/step


2/2 [==============================] - 0s 4ms/step
9900[D loss: 0.6482604444026947, acc.: 60.9375][G loss:0.882538914680481]
2/2 [==============================] - 0s 4ms/step
